In [1]:
import pandas as pd
import os
from sklearn.impute import SimpleImputer
from tqdm.auto import tqdm

/usr/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- Got data from https://traces.cs.umass.edu/index.php/Smart/Smart
- Used 2016 data since it has consistent time gaps
- Convert from minute recordings of kW to kWh by grouping by hour and taking the mean

In [2]:
weather_2015 = pd.read_csv("../data/apartment-weather/apartment2015.csv")
weather_2016 = pd.read_csv("../data/apartment-weather/apartment2016.csv")
weather = pd.concat([weather_2015, weather_2016])
weather = weather.drop(columns=["icon", "summary"])
weather = SimpleImputer(missing_values=pd.NA, strategy='mean').set_output(transform="pandas").fit_transform(weather)
weather['date'] = pd.to_datetime(weather.time, unit='s')
weather = weather.drop(columns="time")
weather

,temperature,humidity,visibility,apparentTemperature,pressure,windSpeed,cloudCover,windBearing,precipIntensity,dewPoint,precipProbability,date
0,11.94,0.76,9.95,2.40,1020.81,5.94,0.03,225.0,0.0,5.78,0.0,2015-01-01 05:00:00
1,13.04,0.73,9.95,3.96,1019.65,5.72,0.04,196.0,0.0,6.14,0.0,2015-01-01 06:00:00
2,15.46,0.69,9.80,7.00,1019.15,5.55,0.10,209.0,0.0,7.16,0.0,2015-01-01 07:00:00
3,16.73,0.65,9.70,6.61,1018.56,7.56,0.06,217.0,0.0,7.04,0.0,2015-01-01 08:00:00
4,16.96,0.62,9.75,6.73,1018.00,7.75,0.06,226.0,0.0,6.33,0.0,2015-01-01 09:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
8779,29.62,0.71,10.00,24.18,1028.14,5.11,0.00,292.0,0.0,21.46,0.0,2017-01-01 00:00:00
8780,26.61,0.78,10.00,26.61,1029.15,1.60,0.00,306.0,0.0,20.81,0.0,2017-01-01 01:00:00
8781,24.42,0.82,10.00,18.56,1030.22,4.64,0.00,299.0,0.0,19.80,0.0,2017-01-01 02:00:00
8782,23.20,0.85,9.89,23.20,1031.12,2.41,0.00,221.0,0.0,19.34,0.0,2017-01-01 03:00:00


In [3]:
data_folder = "../data/apartment/2016"
csv_files = [f for f in os.listdir(data_folder) if ".csv" in f]
client_data = []

for csv_file in tqdm(csv_files):
    df = pd.read_csv(f"{data_folder}/{csv_file}", header=None)
    df.columns = ["date", "consumption"]
    df.date = pd.to_datetime(df.date).dt.floor('h')
    df = df.groupby("date").mean().reset_index()
    full_df = pd.merge(df, weather[weather.date.isin(df.date)])
    client_data.append(full_df.drop(columns="date").to_numpy())

100%|█████████████████████████████████████████████████████████████| 114/114 [00:26<00:00,  4.26it/s]


In [5]:
import pickle

with open("../data/apartment_data.pkl", 'wb') as f:
    pickle.dump(client_data, f)